In [1]:
# We will need the two point library
!pip install twopoint

In [2]:
import sys
sys.path.append("..")
import sacc

In [3]:
# Get an example data file to convert to sacc format.
# This is in the two point format
!wget http://desdr-server.ncsa.illinois.edu/despublic/y1a1_files/chains/2pt_NG_mcal_1110.fits

--2018-11-29 10:15:33--  http://desdr-server.ncsa.illinois.edu/despublic/y1a1_files/chains/2pt_NG_mcal_1110.fits
Resolving desdr-server.ncsa.illinois.edu... 141.142.161.38
Connecting to desdr-server.ncsa.illinois.edu|141.142.161.38|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6600960 (6.3M) [application/octet-stream]
Saving to: ‘2pt_NG_mcal_1110.fits.1’

2pt_NG_mcal_1110.fi 100%[===================>]   6.29M  1.60MB/s    in 5.1s    

2018-11-29 10:15:38 (1.24 MB/s) - ‘2pt_NG_mcal_1110.fits.1’ saved [6600960/6600960]



In [4]:
import twopoint
T = twopoint.TwoPointFile.from_fits("./2pt_NG_mcal_1110.fits")

In [5]:
# Example of making a sacc file

def make_sacc():
    S = sacc.Sacc()
    for kernel in T.kernels:
        for i in range(kernel.nbin):
            tracer_type = 'NZ'
            name = f'{kernel.name}_{i}'
            S.add_tracer(tracer_type, name, kernel.z, kernel.nzs[i])

    # translation from twopoint type codes to sacc type codes:
    table = {
        (twopoint.Types.galaxy_shear_plus_real, twopoint.Types.galaxy_shear_plus_real): 'xi_plus',
        (twopoint.Types.galaxy_shear_minus_real, twopoint.Types.galaxy_shear_minus_real): 'xi_minus',
        (twopoint.Types.galaxy_position_real, twopoint.Types.galaxy_position_real): 'w_theta',
        (twopoint.Types.galaxy_position_real, twopoint.Types.galaxy_shear_plus_real): 'gamma_t',
    }

    for spectrum in T.spectra:
        data_type = table[(spectrum.type1, spectrum.type2)]
        angle_unit = spectrum.angle_unit
        tracer1 = spectrum.kernel1
        tracer2 = spectrum.kernel2
        for i in range(len(spectrum)):
            bin1 = spectrum.bin1[i] - 1
            bin2 = spectrum.bin2[i] - 1
            tracers = [f'{tracer1}_{bin1}', f'{tracer2}_{bin2}']
            value = spectrum.value[i]
            angle = spectrum.angle[i]
            tags = {'scale':angle, 'scale_unit':angle_unit}
            S.add_data_point(data_type, tracers, value, **tags)
    return S

In [6]:

S = make_sacc()


ind1 = S.indices(data_type='xi_plus', tracers=('nz_source_0', 'nz_source_2'))
ind2 = S.indices(data_type='xi_plus', tracers=['nz_source_0', 'nz_source_2'], scale__gt=100.0)
print(ind1)
print(ind2)
S.cut(ind2)
ind3 = S.indices(data_type='xi_plus', tracers=['nz_source_0', 'nz_source_2'])
print(ind3)
S.data[0]
S.tracers['nz_source_0'].z

[40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59]
[56 57 58 59]
Mask the covariance too!
[40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55]


array([0.0051, 0.0151, 0.0251, 0.0351, 0.0451, 0.0551, 0.0651, 0.0751,
       0.0851, 0.0951, 0.1051, 0.1151, 0.1251, 0.1351, 0.1451, 0.1551,
       0.1651, 0.1751, 0.1851, 0.1951, 0.2051, 0.2151, 0.2251, 0.2351,
       0.2451, 0.2551, 0.2651, 0.2751, 0.2851, 0.2951, 0.3051, 0.3151,
       0.3251, 0.3351, 0.3451, 0.3551, 0.3651, 0.3751, 0.3851, 0.3951,
       0.4051, 0.4151, 0.4251, 0.4351, 0.4451, 0.4551, 0.4651, 0.4751,
       0.4851, 0.4951, 0.5051, 0.5151, 0.5251, 0.5351, 0.5451, 0.5551,
       0.5651, 0.5751, 0.5851, 0.5951, 0.6051, 0.6151, 0.6251, 0.6351,
       0.6451, 0.6551, 0.6651, 0.6751, 0.6851, 0.6951, 0.7051, 0.7151,
       0.7251, 0.7351, 0.7451, 0.7551, 0.7651, 0.7751, 0.7851, 0.7951,
       0.8051, 0.8151, 0.8251, 0.8351, 0.8451, 0.8551, 0.8651, 0.8751,
       0.8851, 0.8951, 0.9051, 0.9151, 0.9251, 0.9351, 0.9451, 0.9551,
       0.9651, 0.9751, 0.9851, 0.9951, 1.0051, 1.0151, 1.0251, 1.0351,
       1.0451, 1.0551, 1.0651, 1.0751, 1.0851, 1.0951, 1.1051, 1.1151,
      

In [277]:
len(S.data)

896

In [271]:
ind2

array([56, 57, 58, 59])